# Self streaming data load

In [4]:
import pandas as pd 

In [72]:
history_0 = pd.read_json('StreamingHistory0.json')
history_1 = pd.read_json('StreamingHistory1.json')

In [73]:
df = pd.concat([history_0, history_1])

In [74]:
df.shape

(10769, 4)

In [75]:
df.head()

,endTime,artistName,trackName,msPlayed
0,2020-08-17 19:33,Hz.,yearning,127989
1,2020-08-17 19:35,Flughand,Hylla,130544
2,2020-08-17 19:37,Extra|ordinary,Fable,127989
3,2020-08-17 19:38,Sooda,Exhale,63995
4,2020-08-17 19:41,FloFilz,Dulce,131558


I will convert the listened time, which is like milliseconds, to minutes. 
For a better understanding

In [76]:
df['minPlayed'] = pd.to_datetime(df.msPlayed, unit='ms').dt.minute

Parsing the listened time date to datetime object. To be more manageable

In [77]:
df['endTime'] = pd.to_datetime(df['endTime'])

#### What are the most listened artists?

In [13]:
df.groupby('artistName').agg({'minPlayed': 'sum'}).sort_values(by='minPlayed', ascending=False).head(10)

,minPlayed
artistName,
FKJ,1798
Hot e Oreia,1356
Don L,906
Kali Uchis,750
Matuê,743
"Tyler, The Creator",607
Black Alien,587
Luedji Luna,555
The Internet,552


#### What are the most listened tracks?

In [14]:
df.groupby('trackName').agg({'minPlayed': 'sum'}).sort_values(by='minPlayed', ascending=False).head(10)

,minPlayed
trackName,
Ylang Ylang,423
Easy,410
Summer Nights,356
Risk,346
Brother,329
10 Years Ago,253
Aquela Fé,246
Salve,242
Perdida,230


# Getting track details from Spotify API

In [78]:
import spotipy.util as util
import spotipy
import requests

Instantiating the token and Spotpy lib

In [227]:
username = 'patryckharley'
client_id ='a9dc410f9af04288a5dadb85980b7da6'
client_secret = '1d49736adab64e458b258beeb0396c85'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

sp = spotipy.Spotify(auth=token)

## Getting track and episode IDs

The API has diferent endpoints to podcasts and musics

In [103]:
def get_id(search_name, search_type):
    '''
    Search Spotify content and return her ID
    '''
    try:
        response = sp.search(search_name, type=search_type)
        first_result = response[search_type+'s']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

The search of the API is some slow. So I will create a unique tracks dataframe, for minimize the requests time.

In [104]:
unique_tracks = df.groupby(['artistName', 'trackName'], as_index=False).size()

In [105]:
unique_tracks.shape

(1829, 3)

Applying get_id function for each unique track

In [106]:
unique_tracks['trackId'] = unique_tracks.apply(
    lambda x: 
        get_id(
            x['trackName'] + ' ' + x['artistName'], 
            'track'
        ), 
        axis='columns')

The track which was not finded is a podcast, or does not exist anymore. So let's search the podcast episodes IDs for the null fields

In [125]:
track_isna = unique_tracks['trackId'].isna()

In [111]:
unique_tracks['type'] = ''

In [112]:
unique_tracks['type'][track_isna == False] = 'music'

<ipython-input-112-d646df6825cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_tracks['type'][track_isna == False] = 'music'


In [113]:
unique_tracks['type'][track_isna] = 'podcast'

<ipython-input-113-b8a089ada91b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_tracks['type'][track_isna] = 'podcast'


Applying get_id function for each unique podcast

In [122]:
unique_tracks['trackId'][track_isna] = (
    unique_tracks[track_isna].apply(
        lambda x: 
            get_id(
                x['trackName'] + ' ' + x['artistName'], 
                'episode'
            ), 
        axis='columns'
    )
)

<ipython-input-122-523bff1813d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_tracks['trackId'][track_isna] = (


Now, I will drop the intances who are not track or podcast. They do not exist anymore, I checked in my Spotify app 

In [142]:
unique_tracks = unique_tracks[unique_tracks['trackId'].isna() == False]

In [152]:
unique_tracks.head()

,artistName,trackName,size,trackId,type,artistId
0,((( O ))),iFeel,6,4QAW32QlMwE9TWtI1UFGwf,music,5Lv2GUVwqmQBPwrTrxucE5
1,*NSYNC,Bye Bye Bye,3,62bOmKYxYg7dhrC6gH9vFn,music,6Ff53KvcvAj5U7Z1vojB5o
2,070 Shake,Guilty Conscience - Tame Impala Remix Instrume...,1,2nwc1w2yyOXPCyTaphRQGN,music,12Zk1DFhCbHY6v3xep2ZjI
3,3 Na Massa,Morada Boa Featuring Nina Miranda,1,5x4KSfOm9Bc1ugelLYe4AG,music,1swmzlKgFluITKhxSV9ixX
4,3030,Meu Jardim,1,2unNoJKEtFewEMojsm1i0X,music,3OHpci0ruhvaMv9F795LR5


Here I get the artist IDs, with the same logic of the tracks

In [138]:
unique_tracks['artistId'] = (
    unique_tracks['artistName'].apply(
        lambda x: 
            get_id(x, 'artist')
    )
)

In [149]:
artist_isna = unique_tracks['artistId'].isna()

In [148]:
unique_tracks['artistId'][artist_isna] = (
    unique_tracks[artist_isna]['artistName'].apply(
        lambda x: 
            get_id(x, 'show')
    )
)

<ipython-input-148-87caa60a2217>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_tracks['artistId'][artist_isna] = (
C:\Users\patry\Anaconda3\envs\aula\lib\site-packages\pandas\core\generic.py:8861: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Now I can join the complete streaming history dataframe wiht the unique tracks dataframe

In [154]:
df = df.merge(unique_tracks[['type', 'artistName', 'trackName', 'trackId', 'artistId']])

In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10749 entries, 0 to 10748
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   endTime     10749 non-null  datetime64[ns]
 1   artistName  10749 non-null  object        
 2   trackName   10749 non-null  object        
 3   msPlayed    10749 non-null  int64         
 4   minPlayed   10749 non-null  int64         
 5   type        10749 non-null  object        
 6   trackId     10749 non-null  object        
 7   artistId    10749 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 755.8+ KB


In [158]:
df.to_csv('spotify_history.csv', index=False)

## Getting artist and track details

In [26]:
df = pd.read_csv('spotify_history.csv')

In [159]:
is_track = df['type'] == 'music'

## Artist details

To do not a messy in the streaming history table, I will create a table with just the artists details.

In [175]:
artist_id_list = df[df['type'] == 'music']['artistId'].unique()
artist_list = []
counter = 0

for interval in range(0, len(artist_id_list), 50):
    if len(artist_id_list) - (counter + 50) > 0:
        artist_list += sp.artists(artist_id_list[interval:interval+50])['artists']
    else: 
        artist_list += sp.artists(artist_id_list[interval:])['artists']
    counter += 50    

In [177]:
df_artists = pd.json_normalize(artist_list)

In [179]:
df_artists.head(3)

,genres,href,id,images,name,popularity,type,uri,external_urls.spotify,followers.href,followers.total
0,"[focus beats, lo-fi beats, lo-fi jazzhop]",https://api.spotify.com/v1/artists/59HCGLavl7X...,59HCGLavl7XxBSiAdB2L8j,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Hz.,53,artist,spotify:artist:59HCGLavl7XxBSiAdB2L8j,https://open.spotify.com/artist/59HCGLavl7XxBS...,None,858
1,"[chillhop, focus beats, jazz boom bap, lo-fi b...",https://api.spotify.com/v1/artists/6x5HLaMcoxa...,6x5HLaMcoxaULXpgN0NJbb,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Flughand,55,artist,spotify:artist:6x5HLaMcoxaULXpgN0NJbb,https://open.spotify.com/artist/6x5HLaMcoxaULX...,None,29896
2,[],https://api.spotify.com/v1/artists/0TxC0mgNuAd...,0TxC0mgNuAdLLSVawn3HIW,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Extra|ordinary,44,artist,spotify:artist:0TxC0mgNuAdLLSVawn3HIW,https://open.spotify.com/artist/0TxC0mgNuAdLLS...,None,1398


In [180]:
df_artists.to_csv('artists_details.csv', index=False)

And I'm going to create another table with artist details, but with the genre column exploded. Where will I have each artist replicated for each genre they are part of. That make the analyse in tableau more easy for me

In [214]:
df_genre_artist = pd.json_normalize(artist_list, 'genres', ['id','name', 'popularity', ['followers', 'total'], ['external_urls', 'spotify']])

In [218]:
df_genre_artist.columns = [
    'genre', 'id', 'name', 'popularity', 
    'followers.total', 'external_urls.spotify'
]

In [219]:
df_genre_artist.to_csv('genre_artist_details.csv', index=False)

### Analyse

#### What are the most played genres?

In [222]:
df_genre_artist.groupby('genre').size().sort_values(ascending=False)

genre
nova mpb                    84
brazilian hip hop           76
alternative r&b             75
indie soul                  68
mpb                         59
                            ..
indian singer-songwriter     1
indian indie                 1
hollywood                    1
hip hop tuga                 1
zolo                         1
Length: 491, dtype: int64

## Track details

### Audio Features

Here, I did the same process I did with the artist details.

In [223]:
track_list = df[df['type'] == 'music'].trackId.unique()
features_list = []
counter = 0

for interval in range(0, len(track_list), 100):
    if len(track_list) - (counter + 100) > 0:
        features_list += sp.audio_features(track_list[interval:interval+100])
    else: 
        features_list += sp.audio_features(track_list[interval:])
    counter += 100    

In [233]:
df_features = pd.json_normalize(features_list)

In [248]:
df_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.640,0.4040,0,-6.813,1,0.0560,0.913,0.867,0.139,0.112,74.972,audio_features,6nGs7MpyrZBxQGqP7QZUE5,spotify:track:6nGs7MpyrZBxQGqP7QZUE5,https://api.spotify.com/v1/tracks/6nGs7MpyrZBx...,https://api.spotify.com/v1/audio-analysis/6nGs...,128000,4
1,0.613,0.1420,5,-9.796,0,0.0497,0.859,0.896,0.109,0.472,85.995,audio_features,5225kdD75Gt1JV7aBRcPab,spotify:track:5225kdD75Gt1JV7aBRcPab,https://api.spotify.com/v1/tracks/5225kdD75Gt1...,https://api.spotify.com/v1/audio-analysis/5225...,130547,4
2,0.655,0.3290,8,-11.925,1,0.0592,0.771,0.753,0.111,0.251,149.978,audio_features,0FDfWO6EY4JCJ1qyZV61bj,spotify:track:0FDfWO6EY4JCJ1qyZV61bj,https://api.spotify.com/v1/tracks/0FDfWO6EY4JC...,https://api.spotify.com/v1/audio-analysis/0FDf...,128000,3
3,0.730,0.0268,6,-22.525,1,0.0867,0.444,0.972,0.107,0.817,150.036,audio_features,1WgWROzKGHOXJg141yIV3V,spotify:track:1WgWROzKGHOXJg141yIV3V,https://api.spotify.com/v1/tracks/1WgWROzKGHOX...,https://api.spotify.com/v1/audio-analysis/1WgW...,64008,4
4,0.840,0.4250,9,-9.404,0,0.0871,0.262,0.921,0.116,0.539,92.970,audio_features,4mkjTB0pCowcxXvTDldEOC,spotify:track:4mkjTB0pCowcxXvTDldEOC,https://api.spotify.com/v1/tracks/4mkjTB0pCowc...,https://api.spotify.com/v1/audio-analysis/4mkj...,131559,4


In [240]:
df_features.shape

(1761, 18)

In [ ]:
df_features.to_csv('spotify_audio_features.csv', index=False)

In [4]:
df_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.640,0.4040,0,-6.813,1,0.0560,0.913,0.867000,0.139,0.112,74.972,audio_features,6nGs7MpyrZBxQGqP7QZUE5,spotify:track:6nGs7MpyrZBxQGqP7QZUE5,https://api.spotify.com/v1/tracks/6nGs7MpyrZBx...,https://api.spotify.com/v1/audio-analysis/6nGs...,128000,4
1,0.613,0.1420,5,-9.796,0,0.0497,0.859,0.896000,0.109,0.472,85.995,audio_features,5225kdD75Gt1JV7aBRcPab,spotify:track:5225kdD75Gt1JV7aBRcPab,https://api.spotify.com/v1/tracks/5225kdD75Gt1...,https://api.spotify.com/v1/audio-analysis/5225...,130547,4
2,0.655,0.3290,8,-11.925,1,0.0592,0.771,0.753000,0.111,0.251,149.978,audio_features,0FDfWO6EY4JCJ1qyZV61bj,spotify:track:0FDfWO6EY4JCJ1qyZV61bj,https://api.spotify.com/v1/tracks/0FDfWO6EY4JC...,https://api.spotify.com/v1/audio-analysis/0FDf...,128000,3
3,0.730,0.0268,6,-22.525,1,0.0867,0.444,0.972000,0.107,0.817,150.036,audio_features,1WgWROzKGHOXJg141yIV3V,spotify:track:1WgWROzKGHOXJg141yIV3V,https://api.spotify.com/v1/tracks/1WgWROzKGHOX...,https://api.spotify.com/v1/audio-analysis/1WgW...,64008,4
4,0.840,0.4250,9,-9.404,0,0.0871,0.262,0.921000,0.116,0.539,92.970,audio_features,4mkjTB0pCowcxXvTDldEOC,spotify:track:4mkjTB0pCowcxXvTDldEOC,https://api.spotify.com/v1/tracks/4mkjTB0pCowc...,https://api.spotify.com/v1/audio-analysis/4mkj...,131559,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756,0.759,0.4810,11,-10.414,0,0.0452,0.134,0.051800,0.678,0.116,95.035,audio_features,6lkBUo1eF422uPCrFJrVEQ,spotify:track:6lkBUo1eF422uPCrFJrVEQ,https://api.spotify.com/v1/tracks/6lkBUo1eF422...,https://api.spotify.com/v1/audio-analysis/6lkB...,300722,4
1757,0.840,0.6140,0,-6.908,0,0.0378,0.055,0.000036,0.236,0.967,112.020,audio_features,14zrmCf0dVPaS5iFFUKIK1,spotify:track:14zrmCf0dVPaS5iFFUKIK1,https://api.spotify.com/v1/tracks/14zrmCf0dVPa...,https://api.spotify.com/v1/audio-analysis/14zr...,199443,4
1758,0.607,0.7960,1,-7.066,1,0.0386,0.356,0.928000,0.111,0.236,123.615,audio_features,1hVgSEi1qKBOlACdHGt8vQ,spotify:track:1hVgSEi1qKBOlACdHGt8vQ,https://api.spotify.com/v1/tracks/1hVgSEi1qKBO...,https://api.spotify.com/v1/audio-analysis/1hVg...,363229,4
1759,0.845,0.7380,11,-5.467,0,0.0681,0.310,0.548000,0.111,0.750,121.000,audio_features,0eqFasz9SX1iE9IgyYONhc,spotify:track:0eqFasz9SX1iE9IgyYONhc,https://api.spotify.com/v1/tracks/0eqFasz9SX1i...,https://api.spotify.com/v1/audio-analysis/0eqF...,339174,4


In [6]:
df_features.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000,1761.000000
mean,0.665404,0.577315,5.492902,-8.494314,0.481545,0.136604,0.345087,0.067256,0.195286,0.548777,116.781423,217250.136286,3.950596
std,0.142464,0.189420,3.577086,3.195931,0.499801,0.132631,0.279380,0.196073,0.162686,0.235356,29.096810,66626.657836,0.346001
min,0.192000,0.026800,0.000000,-26.290000,0.000000,0.023200,0.000217,0.000000,0.022600,0.033400,51.293000,23720.000000,1.000000
25%,0.571000,0.457000,2.000000,-10.165000,0.000000,0.044700,0.091400,0.000000,0.098900,0.362000,92.864000,177027.000000,4.000000
50%,0.685000,0.582000,6.000000,-7.978000,0.000000,0.073900,0.276000,0.000027,0.124000,0.555000,115.009000,211138.000000,4.000000
75%,0.774000,0.720000,9.000000,-6.431000,1.000000,0.191000,0.559000,0.003860,0.241000,0.745000,135.982000,249498.000000,4.000000
max,0.971000,0.983000,11.000000,-1.080000,1.000000,0.949000,0.989000,0.972000,0.997000,0.980000,216.109000,610000.000000,5.000000


To visualize the informations as I imagine it, it will be better if I take the features unpivoted. This way, the features I selected will be categories in a single column, with their values in another column.

In [6]:
unpivoted_features = df_features[[
        'id', 'danceability', 'energy', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence'
]].melt(
    id_vars='id', 
    var_name='feature', 
    value_name='value'
)
unpivoted_features

,id,feature,value
0,6nGs7MpyrZBxQGqP7QZUE5,danceability,0.640
1,5225kdD75Gt1JV7aBRcPab,danceability,0.613
2,0FDfWO6EY4JCJ1qyZV61bj,danceability,0.655
3,1WgWROzKGHOXJg141yIV3V,danceability,0.730
4,4mkjTB0pCowcxXvTDldEOC,danceability,0.840
...,...,...,...
14083,6lkBUo1eF422uPCrFJrVEQ,valence,0.116
14084,14zrmCf0dVPaS5iFFUKIK1,valence,0.967
14085,1hVgSEi1qKBOlACdHGt8vQ,valence,0.236
14086,0eqFasz9SX1iE9IgyYONhc,valence,0.750


In [ ]:
unpivoted_features.to_csv('features_mood_unpivot.csv')

### Track details

Here, I did the same process I did with the audio features. After that I create two tables: the first contains only the track details, and the second is a joined table of the tracks details and audio features tables.

In [245]:
track_feature = pd.read_csv('spotify_audio_features.csv')

In [231]:
track_id_list = track_feature['id'].unique()
track_list = []
counter = 0

for interval in range(0, len(track_id_list), 50):
    if len(track_id_list) - (counter + 50) > 0:
        track_list += sp.tracks(track_id_list[interval:interval+50])['tracks']
    else: 
        track_list += sp.tracks(track_id_list[interval:])['tracks']
    counter += 50   

In [236]:
df_track = pd.json_normalize(track_list)

In [260]:
df_track.to_csv('track_details.csv', index=False)

Selecting only the important columns for me

In [239]:
df_track = df_track[
    [
       'duration_ms', 'explicit', 'id', 'name', 'popularity',
       'album.external_urls.spotify', 'album.id',
       'album.name', 'external_urls.spotify'
    ]
]

In [248]:
track_feature.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [254]:
df_feature = track_feature[[
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'id', 'time_signature'
]]

In [255]:
df_track_details = track_feature.merge(df_track, on='id')

In [259]:
df_track_details.head(3)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,duration_ms_x,time_signature,duration_ms_y,explicit,name,popularity,album.external_urls.spotify,album.id,album.name,external_urls.spotify
0,0.640,0.404,0,-6.813,1,0.0560,0.913,0.867,0.139,0.112,...,128000,4,128000,False,yearning,32,https://open.spotify.com/album/5FWI6x6yx5GCzeK...,5FWI6x6yx5GCzeKHUWhvTj,nostalgia,https://open.spotify.com/track/6nGs7MpyrZBxQGq...
1,0.613,0.142,5,-9.796,0,0.0497,0.859,0.896,0.109,0.472,...,130547,4,130547,False,Hylla,37,https://open.spotify.com/album/3MpDoKBQOIuvE1A...,3MpDoKBQOIuvE1AJDD7DPP,peaceful weibs,https://open.spotify.com/track/5225kdD75Gt1JV7...
2,0.655,0.329,8,-11.925,1,0.0592,0.771,0.753,0.111,0.251,...,128000,3,128000,False,Fable,7,https://open.spotify.com/album/6qNQZd5QM0Vy2Uu...,6qNQZd5QM0Vy2UukSykXvM,Fable,https://open.spotify.com/track/0FDfWO6EY4JCJ1q...


In [258]:
df_track_details.to_csv('track_details_features.csv', index=False)